# 발음 평가 전처리
## - Tokenization/Lemmatization
## - Cleaning - 영어,숫자,기호,공백 제거
## - Stopwords 제거


## string data 입력

In [3]:
with open("input.txt") as f:
    input_str = f.read()
print(input_str)

음 아안녕하세요. 전 저는 <name1>입니다. 저는 학생입니다. <name2>대학교 학교에 국 한국어를 배워요. 저는 지금? 좋아요. 어 제가쪽이 음 네 명이에요. 아버지하고 어머니하고 누나하고 저이에요. 아버지는 선생님이에요. 음 아버지는 지금? 좋아요. 어 어머니 어머니는 요리사이에요 예요(이에요). 엄 으음 뜨 뜨라 뜨라 드 드라마를 좋아요. 어 누나 누나 누나가 회사원이에요. 저 저 운도를? 해요. 아주 좋아해요.


## 토큰화(Tokenization)

In [23]:
# !pip3 install PyKomoran

from PyKomoran import *
import pandas as pd

komoran = Komoran("EXP")
input_token_komoran = komoran.get_plain_text(input_str).split(' ')
print('코모란:', input_token_komoran)

df_komoran = pd.DataFrame([x.split('/') for x in input_token_komoran], columns=['형태소', '주석'])

df_komoran

코모란: ['음/IC', '아/IC', '안녕하세요/IC', './SF', '전/MM', '저/NP', '는/JX', '</SS', 'name/SL', '1/SN', '>/SS', '이/VCP', 'ㅂ니다/EF', './SF', '저/NP', '는/JX', '학생/NNG', '이/VCP', 'ㅂ니다/EF', './SF', '</SS', 'name/SL', '2/SN', '>/SS', '대학교/NNG', '학교/NNG', '에/JKB', '국/NNG', '한국어/NNG', '를/JKO', '배우/VV', '어요/EF', './SF', '저/NP', '는/JX', '지금/MAG', '?/SF', '좋/VA', '아요/EF', './SF', '어/IC', '제가/NNP', '쪽/NNB', '이/JKS', '음/IC', '네/MM', '명/NNB', '이/VCP', '에요/EF', './SF', '아버지/NNG', '하/XSV', '고/EC', '어머니/NNG', '하/XSV', '고/EC', '누나/NNG', '하/XSV', '고/EC', '저/NP', '이/VCP', '에요/EF', './SF', '아버지/NNG', '는/JX', '선생/NNG', '님/XSN', '이/VCP', '에요/EF', './SF', '음/IC', '아버지/NNG', '는/JX', '지금/MAG', '?/SF', '좋/VA', '아요/EF', './SF', '어/IC', '어머니/NNG', '어머니/NNG', '는/JX', '요리사/NNG', '이/VCP', '에요/EC', '예/NNG', '이/VCP', '요/EC', '(/SS', '이/VCP', '에요/EF', ')/SS', './SF', '엄/NNP', '으음/IC', '뜨/VV', '뜨/VV', '라/EC', '뜨/VV', '라/EC', '드/NNP', '드라마/NNG', '를/JKO', '좋/VA', '아요/EF', './SF', '어/IC', '누나/NNG', '누나/NNG', '누나/NNG', '가/JKS', '회사원/NNG

,형태소,주석
0,음,IC
1,아,IC
2,안녕하세요,IC
3,.,SF
4,전,MM
...,...,...
122,.,SF
123,아주,MAG
124,좋아하,VV
125,아요,EF


## 정제(Cleaning)
#### - 영어 제거
#### - 숫자 제거
#### - 기호 제거
#### - 공백 제거

In [36]:
import re

# 정규식 패턴
pattern_english = r'^[a-zA-Z]+$'
pattern_number = r'\d+'
pattern_symbol = r'[!@#$%^&*()_+-={}\[\]:;"\'<>,.?/~`|]'
pattern_empty = r'\s+'

# 제거할 형태소 리스트
list_english = []
list_number = []
list_symbol = []
list_empty = []

## 영어 제거
for index, row in df_komoran.iterrows():
    if re.match(pattern_english, row['형태소']):
        list_english.append(index)

df_remove_english = df_komoran.drop(list_english)


## 숫자 제거
for index, row in df_remove_english.iterrows():
    if re.match(pattern_number, row['형태소']):
        list_number.append(index)

df_remove_number = df_remove_english.drop(list_number)


## 기호 제거
for index, row in df_remove_number.iterrows():
    if re.match(pattern_symbol, row['형태소']):
        list_symbol.append(index)

df_remove_symbol = df_remove_number.drop(list_symbol)


## 공백 제거
for index, row in df_remove_symbol.iterrows():
    if re.match(pattern_empty, row['형태소']):
        list_empty.append(index)

df_data = df_remove_symbol.drop(list_empty)

df_data['형태소'].unique()


array(['음', '아', '안녕하세요', '전', '저', '는', '이', 'ㅂ니다', '학생', '대학교', '학교',
       '에', '국', '한국어', '를', '배우', '어요', '지금', '좋', '아요', '어', '제가', '쪽',
       '네', '명', '에요', '아버지', '하', '고', '어머니', '누나', '선생', '님', '요리사',
       '예', '요', '엄', '으음', '뜨', '라', '드', '드라마', '가', '회사원', '운도', '아주',
       '좋아하'], dtype=object)

#### 미사용 형태 주석 제거

In [41]:
## 미사용 형태 주석 제거
using_pos = ['NP', 'NNG', 'XSA', 'EP', 'EF', 'JX', 'NNP', 'JKB', 'XSV', 'EC', 'VX', 'ETM', 'XSN', 'VCP', 'VV', 'VA', 'JKS', 'MAG', 'XR', 'JC', 'NA', 'VCN', 'MM', 'ETN', 'MAJ', 'JKG', 'JKC', 'IC', 'NNB', 'NR', 'JKO', 'XPN', 'JKQ']

df_data = df_data[df_data['주석'].isin(using_pos)]

print(df_data['주석'].unique())

['IC' 'MM' 'NP' 'JX' 'VCP' 'EF' 'NNG' 'JKB' 'JKO' 'VV' 'MAG' 'VA' 'NNP'
 'NNB' 'JKS' 'XSV' 'EC' 'XSN' 'VX']


## 불용어(Stopwords) 제거

In [43]:
## stopwords 리스트 생성
txt_tmp = open("./koreanStopwords.txt", 'r', encoding="utf-8")

stopwords = []
while True:
    line = txt_tmp.readline()
    if not line: break
    form, pos, _ = line.split()
    stopwords.append([form, pos])

txt_tmp.close()

## stopwords 제거
for f, p in stopwords:
    df_data = df_data[~((df_data["형태소"] == f) & (df_data["주석"] == p))]

df_data

,형태소,주석
0,음,IC
1,아,IC
2,안녕하세요,IC
4,전,MM
5,저,NP
...,...,...
120,하,VX
121,아요,EF
123,아주,MAG
124,좋아하,VV
